In [13]:
import pandas as pd
import numpy as np

In [14]:
files= ['cancer_data.csv', 'respiratory_data.csv', 'cardiovascular_data.csv']

In [15]:
#Function to calculate months between dates 
def calculate_months(row):
    if row['Start Date'] == 'NaT' or row['Completion Date'] == 'NaT' :
        val = np.nan
    else:
        val = 12 * (row['Completion Date'].year - row['Start Date'].year) + (row['Completion Date'].month - row['Start Date'].month)
    return val

In [16]:
for file in files: 
    #read data file 
    data = pd.read_csv(file)
    
    #extract type of disease 
    disease= file.split('_')[0]
    
    #read enrollment for specific disease 
    enrollment= pd.read_csv('enrollment_countries_' + disease + '.csv')
    
    #convert NCT Number to int 
    enrollment['NCT Number'] = enrollment['NCT Number'].str[3:].astype('int')
    data['NCT Number'] = data['NCT Number'].str[3:].astype('int')
    
    #set NCT number as index and join the datas 
    my_df= data.set_index('NCT Number').join(enrollment.set_index('NCT Number'))
    
    #Replace Actual values that are not provided by NaN
    my_df['Actual'] = my_df['Actual'].replace('Not Provided',np.NaN)
    
    #Convert Actual to float
    my_df['Actual'] = my_df['Actual'].astype('float')
    
    #Replace Estimated values that are equal to the actual to the actual 
    my_df['Estimated'] = np.where(my_df['Estimated'] == 'Same as current',my_df['Actual'], my_df['Estimated'])
    
    #Replace Estimated values that are not provided by NaN
    my_df['Estimated']= my_df['Estimated'].replace('Not Provided',np.NaN)
    
    #Convert Estimated to float
    my_df['Estimated'] = my_df['Estimated'].astype('float')
    
    #Create estimated  fraction  (EFraction)
    my_df['EFraction']= my_df['Actual']/my_df['Estimated']
    
    #Replace Phases that are not applicable by NaN
    #my_df['Phases'] = my_df['Phases'].replace('Not Applicable',np.NaN ) 
    
    #Change Phases 
    
    phases_dict= {'Not Applicable':0, 'Phase 1' : 1, 'Phase 2' :2 , 'Phase 1|Phase 2' : 1.5,
       'Phase 4' : 4 , 'Phase 3': 3 , 'Phase 2|Phase 3' : 2.5, 'Early Phase 1' : 0.5} 
    my_df =my_df.replace({"Phases": phases_dict})
    
    #Change Status 
    status_dict = {"Terminated" : 0 , "Completed" : 1}
    my_df =my_df.replace({"Status": status_dict})
    
    #Split Countries  and change to NaN if not provided 
    my_df['Countries']= my_df['Countries'].replace('Not Provided',np.NaN)
    my_df["Countries"]= my_df["Countries"].str.split(',')
    
        #Create intervention and change to numeric
    my_df["Intervention"]= my_df["Interventions"].str.split(': ').str[0]
    intervention_dict = {'Drug': 0, 'Behavioral' : 1, 'Radiation': 2, 'Procedure':3, 'Dietary Supplement':4,
                     'Biological':5, 'Diagnostic Test':6, 'Device':7,'Genetic':8, 'Combination Product':9,
                     'Other': 10}
    my_df =my_df.replace({"Intervention": intervention_dict})
    
    #Change genders
    gender_dict= {'Female': 0, 'Male' : 1, 'All': 2}
    my_df = my_df.replace({'Gender': gender_dict})
    
    #Create Age_Type column
    my_df['Age_Type'] = my_df['Age'].apply(lambda st: st[st.find("(")+1:st.find(")")]).copy()
    age_dict = {'Adult, Older Adult' : 3  , 'Older Adult': 4 , 'Adult' : 2,
       'Child, Adult, Older Adult' : 5 , 'Child, Adult': 1 ,
       'Child, Adult, Older Adul':5  , 'Child' : 0} 
    my_df = my_df.replace({'Age_Type': age_dict})
    
    #Create Disease column
    my_df['Disease']= disease
    
    #Change dates to month periods 
    my_df['Start Date'] = pd.to_datetime(my_df['Start Date'], errors='coerce').dt.to_period('M')
    my_df['Completion Date'] = pd.to_datetime(my_df['Completion Date'], errors='coerce').dt.to_period('M')
    
    #Create Months column (amount of months from start date to end date of phase)
    my_df['Months'] = my_df.apply(calculate_months, axis=1)
    
        
    #replaces infinity with NaN
    my_df.replace([np.inf, -np.inf], np.nan, inplace=True)
    # Drop rows with NaN
    my_df.dropna(inplace=True)

    #my_df.to_csv( disease + '_data_cleaned.csv',index=False)
    my_df.to_pickle(disease + '_data_cleaned.pkl') 